In [ ]:
## project for VU winterschool scraping HBR articles on coaching and use NLP to look at topic modelling 

## This is the 'parent' link: https://hbr.org/search?N=0+4294967064&Ntt=coaching&loaded=1 that contains all other links in the 
## headings

## first I need to write a scraper that scrapes all the article header links and safe these in a list. Then I need to write
## a loop for extracting the text for each article in that list.

## example Xpath: //*[@id="main"]/div[5]/search-stream/div/div/div/div[5]/stream-list/ul/li/div/div/stream-item/h3/a


In [1]:
## importing the libraries 

from lxml import html
from lxml.html import fromstring
import requests
import re
from bs4 import BeautifulSoup
import urllib.request
import csv
import time 
import random
import pandas as pd
import numpy as np 
import nltk
from nltk.sentiment import vader
from nltk.corpus import stopwords
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/timtheeboom/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timtheeboom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/timtheeboom/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/timtheeboom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/timtheeboom/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
## setting the base URL 

baseurl = "https://hbr.org/search?N=0+4294967064&Ntt=coaching&loaded=100"

myurl = baseurl
htmlsource = requests.get(myurl).text
tree = html.fromstring(htmlsource)

In [3]:
## create alist of the links based on the xpath (found by using the console)

articles = tree.xpath('//*[@class= "hed"]/a/@href')

In [4]:
## check the list 

for item in articles:
    print(item)

/product/hbr-guide-to-coaching-employees-ebook-tools/13990H?sku=13990H-KND-ENG
/2019/11/how-technology-is-transforming-executive-coaching
/2018/10/how-peer-coaching-can-make-work-less-lonely
/2018/07/4-signs-an-executive-isnt-ready-for-coaching
/2017/09/what-i-learned-about-coaching-after-losing-the-ability-to-speak
/2017/07/when-coaching-finds-that-an-executive-isnt-in-the-right-role
/2017/07/younger-and-older-executives-need-different-things-from-coaching
/2016/11/patients-make-better-medical-choices-with-coaching
/2016/09/every-manager-needs-to-practice-two-types-of-coaching
/2016/05/for-delegation-to-work-it-has-to-come-with-coaching
/2016/01/how-mindfulness-improves-executive-coaching
/2016/01/strengths-based-coaching-can-actually-weaken-you
/2015/05/4-reasons-managers-should-spend-more-time-on-coaching
/2015/05/a-high-percentage-move-to-increase-revenue
/2015/04/overcoming-the-toughest-common-coaching-challenges
/2015/03/quiz-do-you-know-when-coaching-works-and-when-it-doesnt
/20

In [5]:
## check the length of the list 

len(articles)

1001

In [34]:
## concatonate every item in the list with the base url "https://hbr.org/" using list comprehension so that it can be used for scraping

baselink = "https://hbr.org/"

allLinks = [baselink + item for item in articles]

for link in all_links: 
    print(link)
    

https://hbr.org//product/hbr-guide-to-coaching-employees-ebook-tools/13990H?sku=13990H-KND-ENG
https://hbr.org//2019/11/how-technology-is-transforming-executive-coaching
https://hbr.org//2018/10/how-peer-coaching-can-make-work-less-lonely
https://hbr.org//2018/07/4-signs-an-executive-isnt-ready-for-coaching
https://hbr.org//2017/09/what-i-learned-about-coaching-after-losing-the-ability-to-speak
https://hbr.org//2017/07/when-coaching-finds-that-an-executive-isnt-in-the-right-role
https://hbr.org//2017/07/younger-and-older-executives-need-different-things-from-coaching
https://hbr.org//2016/11/patients-make-better-medical-choices-with-coaching
https://hbr.org//2016/09/every-manager-needs-to-practice-two-types-of-coaching
https://hbr.org//2016/05/for-delegation-to-work-it-has-to-come-with-coaching
https://hbr.org//2016/01/how-mindfulness-improves-executive-coaching
https://hbr.org//2016/01/strengths-based-coaching-can-actually-weaken-you
https://hbr.org//2015/05/4-reasons-managers-should-

In [18]:
## extract the article text a link (course approach)

response = requests.get("https://hbr.org//2018/10/how-peer-coaching-can-make-work-less-lonely")
text = response.text
print(text)
    



<!DOCTYPE html>
<!--[if IE 8]>
<html class="ie8 ">
<![endif]-->
<!--[if IE 9]>
<html class="ie9 ">
<![endif]-->
<!--[if !IE]><!-->
<html class="">
<!--<![endif]--><!--
 _____________    _    _                       hbr.org _ 
|/////////////|  | |  | |                             | |
|:\/E:R|:T/\S:|  | |__| | __ _ _ ____   ____ _ _ __ __| |
|+++++++++++++|  |  __  |/ _` | '__\ \ / / _` | '__/ _` |
|\:+:\   /:+:/|  | |  | | (_| | |   \ V / (_| | | | (_| |
| \:::\ /:::/ |  |_|__|_|\__,_|_| _  \_/ \__,_|_|  \__,_|
\  \ ::+:: /  /  |  _ \          (_)                     
 \ /::/ \::\ /   | |_) |_   _ ___ _ _ __   ___  ___ ___  
  \+:/   \:+/    |  _ <| | | / __| | '_ \ / _ \/ __/ __| 
   \/_____\/     | |_) | |_| \__ \ | | | |  __/\__ \__ \ 
To improve the   |____/ \__,_|___/_|_| |_|\___||___/___/ 
practice of      |  __ \          (_)                    
management and   | |__) |_____   ___  _____      __      
its impact in a  |  _  // _ \ \ / / |/ _ \ \ /\ / /      
changing world.  | |

In [ ]:
## extract text based on content-area--article column class using the Beautifull Soup Library 

r = requests.get("https://hbr.org//2018/10/how-peer-coaching-can-make-work-less-lonely")
html_text = r.content
soup = BeautifulSoup(html_text, 'html.parser') 

mainarticle = soup.find_all('div', attrs={'class':'content-area--article column'})

## now get all the text within mainarticle 

mainarticletext = [i.get_text() for i in mainarticle]


In [96]:
## testen met subset van de links 

subsetLinks = allLinks[0:5]
subsetCorpusDictList = []
subsetCorpusTitlePlusTextList = []

for link in subsetLinks:
    time.sleep(random.randrange(1,2)) ## add a random wait intervall while the loop is not finished, to prevent HBR from blocking me
    source = urllib.request.urlopen(link).read() ## bs way of reading the links
    soup = BeautifulSoup(source,'lxml') ## creating a beautifull soup 
    articleText = soup.find_all('div', attrs={'class':'content-area--article column'}) ## extracting text based on div class
    articleTitle = soup.find_all('h1', attrs={'class':'article-hed'}) ## extracting text based on div class
    articleTextInner = [i.get_text() for i in articleText] ## using list comprehension to extract inner html 
    articleTextInner = [articleTextInner[0].replace("\n","").replace("\t","").replace("—\xa0","") for i in articleTextInner] ## using list comprehension to further clean the text
    articleTitleInner = [i.get_text() for i in articleTitle] ## using list comprehension to extract inner html 
    articleTitleInner = [articleTitleInner[0].replace("\n","").replace("\t","").replace("—\xa0","") for i in articleTitleInner] ## using list comprehension to further clean the text
    result = {"title":articleTitleInner, "text":articleTextInner} ## saving result to dictionary 
    subsetCorpusDictList.append(result) ## creating a list of dinctionaries that makes it easier to export to CSV  

In [97]:
## checking the results 
for i in subsetCorpusDictList:
    print("\n")
    print(i)
    print("\n")



{'title': [], 'text': []}




{'title': ['How Technology Is Transforming Executive Coaching'], 'text': ["googletag.cmd.push(function() { googletag.display('DFP_TITLE_pos2'); });Loading...Executive SummaryCoaches have always sought to help their clients improve. Moving forward, strategically applying technology — alongside their own judgement, warmth, and integrity — will be an increasingly important way for them to do so. The authors describe four key areas where technology can transform the act — and the impact — of coaching: 1) Technology can help monitor progress towards goals against a clear baseline. 2) It can build a richer picture of what the client is saying (and not saying). 3) It can develop options based on scenarios, simulations, and extrapolations. 4) It can use “nudges” to encourage and reinforce target behaviors.   Mimi Haddon/Getty Images  Years ago, executive coaching was stigmatized as “remedial help for underperformers.” More recently, it’s transformed into an elit

In [ ]:
## writing results to CSV 

keys = subsetCorpusDictList[0].keys() ## extract keys from first entry in subset_corpus_list from https://stackoverflow.com/questions/43138628/convert-list-of-dict-to-csv-in-python
    with open('test.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(subsetCorpusDictList)

In [98]:
## creating a dataframe from the dictionary to that i can use the apply function to further preprocess the data  

subsetCorpusDf = pd.DataFrame(subsetCorpusDictList)
subsetCorpusDf

,title,text
0,[],[]
1,[How Technology Is Transforming Executive Coac...,[googletag.cmd.push(function() { googletag.dis...
2,[How Peer Coaching Can Make Work Less Lonely],[googletag.cmd.push(function() { googletag.dis...
3,[4 Signs an Executive Isn’t Ready for Coaching],[googletag.cmd.push(function() { googletag.dis...
4,[What I Learned About Coaching After Losing th...,[googletag.cmd.push(function() { googletag.dis...


In [123]:
## further preprocessing: removing punctuation 

noPuncTextList = []

for text in articleTextInner:
    result = re.sub(r'[^\w\s]','',text)
    noPuncTextList.append(result)
    

In [ ]:
## further preprocessing: removing stopwords




